#  Chatbot Using Python, NLTK, and TensorFlow

# Import the Required Libraries

In [3]:
import json
import pickle
import random
import nltk
from nltk.stem import WordNetLemmatizer
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Dense,Activation,Dropout
import matplotlib.pyplot as plt
from matplotlib import rcParams

#  Load the Data

In [4]:
nltk.download('omw-1.4')
with open(r'intents.json') as data:
    intents = json.loads(data.read())
print(intents)

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


{'intents': [{'tag': 'greeting', 'patterns': ['Hi there', 'How are you', 'Is anyone there?', 'Hey', 'Hola', 'Hello', 'Good day'], 'responses': ['Hello, thanks for asking', 'Good to see you again', 'Hi there, how can I help?'], 'context': ['']}, {'tag': 'goodbye', 'patterns': ['Bye', 'See you later', 'Goodbye', 'Nice chatting to you, bye', 'Till next time'], 'responses': ['See you!', 'Have a nice day', 'Bye! Come back again soon.'], 'context': ['']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", 'Awesome, thanks', 'Thanks for helping me'], 'responses': ['Happy to help!', 'Any time!', 'My pleasure'], 'context': ['']}, {'tag': 'noanswer', 'patterns': [], 'responses': ["Sorry, can't understand you", 'Please give me more info', 'Not sure I understand'], 'context': ['']}, {'tag': 'options', 'patterns': ['How you could help me?', 'What you can do?', 'What help you provide?', 'How you can be helpful?', 'What support is offered'], 'responses': ['I can guide you through

# Tokenization

In [5]:
words,classes,documents = [],[],[]
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word = nltk.word_tokenize(pattern)
        words.extend(word)
        classes.append(intent['tag'])
        documents.append((word,intent['tag']))
#print(documents)  
        
print(classes)


['greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'greeting', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'goodbye', 'thanks', 'thanks', 'thanks', 'thanks', 'thanks', 'options', 'options', 'options', 'options', 'options', 'adverse_drug', 'adverse_drug', 'adverse_drug', 'adverse_drug', 'adverse_drug', 'blood_pressure', 'blood_pressure', 'blood_pressure', 'blood_pressure', 'blood_pressure', 'blood_pressure_search', 'blood_pressure_search', 'blood_pressure_search', 'blood_pressure_search', 'blood_pressure_search', 'pharmacy_search', 'pharmacy_search', 'pharmacy_search', 'pharmacy_search', 'pharmacy_search', 'hospital_search', 'hospital_search', 'hospital_search', 'hospital_search', 'hospital_search']


#  Lemmatization

In [6]:
lemmatizer = WordNetLemmatizer()
ignore_w = ['!','?']
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_w]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
pickle.dump(words,open('words.pkl','wb'))
pickle.dump(classes,open('classes.pkl','wb'))
#print(words)
print(classes)



['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']


#  Create Data for Training

In [7]:
training = []
out = [0]*len(classes)
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [lemmatizer.lemmatize(w.lower())for w in pattern_words]
    for w in words:
        if w in pattern_words:
            bag.append(1)
        else:
            bag.append(0)
    
    output_row = list(out)
    output_row[classes.index(doc[1])] = 1
    training.append([bag,output_row])
random.shuffle(training)
value = int(0.7*len(training))
training = np.array(training,dtype = 'object')
print(training)
x_train = list(training[:value,0])
y_train = list(training[:value,1])
x_test = list(training[value:,0])
y_test = list(training[value:,1])

    


[[list([0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1])
  list([0, 0, 0, 0, 1, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0])
  list([0, 0, 0, 0, 0, 1, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0])
  list([0, 1, 0, 0, 0, 0, 0, 0, 0])]
 [list([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

#  Design the Model

In [8]:
model = Sequential()
model.add(Dense(128, input_shape = (len(x_train[0]),),activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(64,activation = 'relu'))
model.add(Dropout(0.5))
model.add(Dense(len(y_train[0]),activation = 'softmax'))

#  Train and Save the Model

In [9]:
sgd = SGD(learning_rate = 0.01, momentum = 0.9, nesterov = True)
model.compile(loss = 'categorical_crossentropy',optimizer = sgd,metrics = ['accuracy'])
hist = model.fit(np.array(x_train),np.array(y_train),epochs = 200,batch_size = 5,validation_data = (x_test,y_test),verbose = 1)
model.save('trained_moded.h5',hist) 


Epoch 1/200
7/7 [==============================] - 1s 91ms/step - loss: 2.2529 - accuracy: 0.0938 - val_loss: 2.1781 - val_accuracy: 0.2667
Epoch 2/200
7/7 [==============================] - 0s 12ms/step - loss: 2.1284 - accuracy: 0.2188 - val_loss: 2.1658 - val_accuracy: 0.2667
Epoch 3/200
7/7 [==============================] - 0s 13ms/step - loss: 2.0898 - accuracy: 0.2812 - val_loss: 2.1540 - val_accuracy: 0.2667
Epoch 4/200
7/7 [==============================] - 0s 13ms/step - loss: 2.0205 - accuracy: 0.2188 - val_loss: 2.1308 - val_accuracy: 0.2667
Epoch 5/200
7/7 [==============================] - 0s 13ms/step - loss: 1.9785 - accuracy: 0.3125 - val_loss: 2.1153 - val_accuracy: 0.2667
Epoch 6/200
7/7 [==============================] - 0s 13ms/step - loss: 2.0039 - accuracy: 0.3125 - val_loss: 2.1052 - val_accuracy: 0.2667
Epoch 7/200
7/7 [==============================] - 0s 13ms/step - loss: 1.8816 - accuracy: 0.2812 - val_loss: 2.0914 - val_accuracy: 0.2667
Epoch 8/200
7/7 [===

#  Print the training curves

In [10]:
plt.rcParams['figure.figsize']  = (12,8)
n = np.arange(0,200)
plt.style.use('ggplot')
plt.figure()
plt.plot(n,hist.history['loss'],label = 'train_loss')
plt.plot(n,hist.history['val_loss'], label = 'val_loss')
plt.plot(n,hist.history['accuracy'],label = 'accuracy')
plt.plot(n,hist.history['val_accuracy'],label = 'val_accuracy')
plt.title("Training loss and accuracy")
plt.xlabel("Epoch #")
plt.legend()


<Figure size 1200x800 with 0 Axes>

<Figure size 1200x800 with 0 Axes>